In [1]:
from __future__ import division

In [2]:
import os
import random
import math
import datetime
import time

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
import numpy as np
import pandas as pd

In [5]:
import tensorflow as tf
from keras import backend as K
from keras import regularizers

Using TensorFlow backend.


In [6]:
%load_ext autoreload

%autoreload 1

%aimport training

In [44]:
ds = (training.DataLoader()
      .add_dataset('random-2017-10-21-13:41:47', 100)
      .add_dataset('gen-1-cov2d_beta_2017_10_22_142925', 200)
      .load())

In [ ]:
def train(graph, output_prefix,
          features_rain, targets_train, features_test, targets_test,
          batch_size, num_batches, batch_how='iter'):
    
    # To be run, a graph must have the following:
    # - Tensor: board:0
    # - Tensor: outcome:0
    # - Tensor: Merge/MergeSummary:0
    # - Operation: init_op
    # - Operation: training/train_step

    board = graph.get_tensor_by_name('board:0')
    outcome = graph.get_tensor_by_name('outcome:0')
    all_summaries = graph.get_tensor_by_name('Merge/MergeSummary:0')
    
    init_op = graph.get_operation_by_name('init')
    train_step = graph.get_operation_by_name('training/train_step')
    
    with tf.Session(graph=graph) as sess:
    
        K.set_session(sess)
        sess.run(init_op)

        train_writer = tf.summary.FileWriter(output_prefix, sess.graph)
        
        print "Running {}".format(output_prefix)
            
        t = time.time()
        delta_t = 0
        
        for i in range(num_batches):
        
            if i % 1000 == 0:
                
                delta_t = time.time() - t
                t = time.time()
                    
                acc = acc_value.eval(feed_dict={board: features_test, outcome: targets_test}).mean()
                print "Batch {} Hold-Out Accuracy: {} Time taken: {:.1f}s".format(i, acc, delta_t)

                summary = sess.run(all_summaries, feed_dict={board: features_test, outcome: targets_test})
                train_writer.add_summary(summary, i)            
            
            batch = get_batch(batch_size, i, [features_train, targets_train], how=batch_how)        
            train_step.run(feed_dict={board: batch[0], outcome: batch[1]})
        
        print "DONE TRAINING"
        print "FINAL ACCURACY: {}".format(acc)
        train_writer.close()
        
        model_dir = '{}/model'.format(output_prefix)
        print "SAVING TO: {}".format(model_dir)
        tf.train.Saver().save(sess, model_dir)

# Gen 1

In [7]:
# 100,000 rows of random vs random (gen 0) data
key = 'random-2017-10-21-13:41:47'

features, targets, features_train, targets_train, features_test, targets_test = training.load_data('training_data/{}'.format(key))

In [ ]:
print len(features_train), len(features_test)
print len(targets_train), len(targets_test)

print targets_test.win.value_counts(normalize=True).loc[1]
print targets_test.lose.value_counts(normalize=True).loc[1]
print targets_test.draw.value_counts(normalize=True).loc[1]

## Attempt 1: Normal dense fully-connected neural network

In [62]:
from keras.layers import Dense
from keras.objectives import categorical_crossentropy
from keras.metrics import categorical_accuracy as accuracy
from keras.regularizers import l2

# Create a graph to hold the model.
graph = tf.Graph()

# Create model in the graph.
with graph.as_default():

    # Keras layers can be called on TensorFlow tensors:
    board = tf.placeholder(tf.float32, shape=(None, 42), name='board')
    outcome = tf.placeholder(tf.float32, shape=(None, 3), name='outcome')

    # Fully connected layers
    
    x = Dense(512,
              activation='tanh',
              kernel_regularizer=regularizers.l2(0.1),
              bias_regularizer=regularizers.l2(0.1),
              kernel_initializer='random_uniform',
              bias_initializer='zeros')(board)
    
    x = Dense(512,
              activation='tanh',
              kernel_regularizer=regularizers.l2(0.1),
              bias_regularizer=regularizers.l2(0.1),
              kernel_initializer='random_uniform',
              bias_initializer='zeros')(x)
   
    x = Dense(512,
              activation='tanh',
              kernel_regularizer=regularizers.l2(0.1),
              bias_regularizer=regularizers.l2(0.1),
              kernel_initializer='random_uniform',
              bias_initializer='zeros')(x)
    
    x = Dense(48,
              activation='tanh',
              kernel_regularizer=regularizers.l2(0.1),
              bias_regularizer=regularizers.l2(0.1),
              kernel_initializer='random_uniform',
              bias_initializer='zeros')(x)

    # output layer with 10 units and a softmax activation
    preds = Dense(3, activation='softmax', name='preds')(x) 
        
    with tf.name_scope('evaluation') as scope:
        loss = tf.reduce_mean(categorical_crossentropy(outcome, preds), name='loss')
        tf.summary.scalar('loss', loss)    
    
        acc_value = tf.identity(accuracy(outcome, preds), name='accuracy')
        tf.summary.scalar('accuracy', tf.reduce_mean(acc_value))
    
    with tf.name_scope('training') as scope:
        train_step = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(loss, name='train_step')
    
    # Initialize all variables
    init_op = tf.global_variables_initializer()
    
    all_summaries = tf.summary.merge_all()

In [63]:
graph.get_tensor_by_name('evaluation/accuracy:0')

<tf.Tensor 'evaluation/accuracy:0' shape=(?,) dtype=float32>

In [64]:
acc_value

<tf.Tensor 'evaluation/accuracy:0' shape=(?,) dtype=float32>

In [43]:
ds.y_test.shape

(952, 5)

In [ ]:
training.train(graph, './models/dense_{}'.format(datetime.datetime.now().strftime("%Y_%m_%d_%H%M%S")),
               ds,  batch_size=200, num_batches=2000)

Running ./models/dense_2017_10_28_140617
Batch 000000 Hold-Out Accuracy: 0.3590 Loss: 1.0976 Time taken: 0.0s
Batch 001000 Hold-Out Accuracy: 0.4832 Loss: 0.9314 Time taken: 15.6s


In [ ]:
train(graph, './models/dense_{}'.format(datetime.datetime.now().strftime("%Y_%m_%d_%H%M%S")),
      features_train, targets_train, features_test, targets_test,
      batch_size=200, num_batches=2000)

## Attempt 2: More complicated covnets

In [ ]:
from keras.layers import Conv2D, Flatten
from keras.layers.core import Reshape
from keras.layers.merge import Concatenate

# Create a graph to hold the model.
graph = tf.Graph()

# Create model in the graph.
with graph.as_default():
    
    # Keras layers can be called on TensorFlow tensors:
    board = tf.placeholder(tf.float32, shape=(None, 42), name='board') 
    outcome = tf.placeholder(tf.float32, shape=(None, 3), name='outcome')    
    
    dense_args = dict(
        use_bias=True,
        activation='relu',
        kernel_initializer='random_uniform',
        bias_initializer='zeros',
        kernel_regularizer=regularizers.l2(0.01),
        bias_regularizer=regularizers.l2(0.01),    
    )
    
    d1 = Dense(512, **dense_args)(board)
        
    # The input data is [col0=[row_0, row_1, ...], col1=[row_0, row_1], ...]
    rs = Reshape((7, 6, 1), input_shape=(42,))(board)
    
    conv_args = dict(
        use_bias=True,
        activation='relu',
        kernel_initializer='random_uniform',
        bias_initializer='zeros',
        kernel_regularizer=regularizers.l2(0.01),
        input_shape=(7, 6, 1),
        padding='valid'
    )
    
    # We use a few parallel covents, that we combine in the end        
    ca = Flatten()(Conv2D(8, kernel_size=(5, 5), **conv_args)(rs))
    cb = Flatten()(Conv2D(8, kernel_size=(4, 4), **conv_args)(rs))
    cc = Flatten()(Conv2D(4, kernel_size=(1, 4), **conv_args)(rs))
    cd = Flatten()(Conv2D(4, kernel_size=(4, 1), **conv_args)(rs))

    merged = Concatenate()([d1, ca, cb, cc, cd])
    
    x = Dense(256,  **dense_args)(merged)         
    x = Dense(128,  **dense_args)(x)    
    x = Dense(12,   **dense_args)(x)
    
    # output layer with 10 units and a softmax activation
    preds = Dense(3, activation='softmax', name='preds')(x) 
    
    with tf.name_scope('evaluation') as scope:
        loss = tf.reduce_mean(categorical_crossentropy(outcome, preds))
        tf.summary.scalar('loss', loss)
        
        acc_value = accuracy(outcome, preds)
        tf.summary.scalar('accuracy', tf.reduce_mean(acc_value))
    
    with tf.name_scope('training') as scope:
        train_step = tf.train.AdamOptimizer(learning_rate=0.01).minimize(loss)    
    
    # Initialize all variables
    init_op = tf.global_variables_initializer()
    
    all_summaries = tf.summary.merge_all()

In [ ]:
sess = run(graph,
           './models/gen1-cov2d_beta_{}'.format(datetime.datetime.now().strftime("%Y_%m_%d_%H%M%S")),
           batch_size=500,
           num_batches=20000)

# Gen 2

In [ ]:
# Use all the advanced data
# 10,000 rows of gen-1 vs gen-1 data
key = 'gen-1-cov2d_beta_2017_10_22_142925'
features, targets, features_train, target_train, features_test, target_test = load_data('training_data/gen-1-cov2d_beta_2017_10_22_142925')

In [ ]:
targets.win.value_counts(normalize=True)